In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import sys, os

In [21]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [4]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [57]:
# searched_sentence = 'information about empower users'
# searched_sentence = 'information about empower employees'
# searched_sentence = 'translators worked hours'
# searched_sentence = 'how much translators worked'
# searched_sentence = 'data about translators work efficiency'
# searched_sentence = 'cant see productivity data'
# searched_sentence = 'productive hours worked not visible'
# searched_sentence = 'data about vendors'
searched_sentence = 'data about suppliers'

# sentences = [
#     'data about helix purchases', 'data about empower transactions', 'data about helix jobs',
#     'data about employees names', 'information about workers surnames', 'data about employees names',
#     'data about empower purchases', 'data about helix procurements'
# ]
sentences = pd.read_excel(os.path.join(os.path.dirname(os.getcwd()), 'data', 'documentation.xlsx')).values.flatten()
sen_similarities = pd.DataFrame()

for sentence in sentences:
    similarity = cos_sim(model([sentence])[0], model([searched_sentence])[0])
    new_row = pd.DataFrame([[searched_sentence, sentence, similarity]])
    sen_similarities = pd.concat((sen_similarities, new_row))
        
sen_similarities.reset_index(inplace = True, drop = True)
sen_similarities.columns = ['searched_sentence', 'sentence', 'similarity_score']
sen_similarities = sen_similarities.sort_values(by = 'similarity_score', ascending = False)
sen_similarities.reset_index(drop = True, inplace = True)

In [59]:
for i, sentence in enumerate(sen_similarities.loc[:, 'sentence']):
    print(f'{i}) ', sentence, '\n')

0)  in order to allow someone to see someone's productivity data we need to adjust data security rules in elasticube 

1)  data about how much productive hours or words someone has is taken from the table stage.emp.tbl_TSProductivityTimeSpans and stage.emp.tbl_TSProductivityEntries 

2)  table Stage.emp.tblLU_CostControlCodes contains information about cost codes 

3)  table Stage.emp.tblLU_CostCenters contains information about cost centers 

4)  in order to prepare data for headcount dashboard we need to use ssis package Corpsql12>DBI Manual Updates/HEADCOUNT_NEW which creates table Stage.dbi.Headcount_NEW which is used in elasticube 

5)  table stage.dbi.Headcount_NEW contains information about employees who are working for us, who joined us and who leaved us 

6)  table stage.helix.JT_External_PO_v2 contains information about purchase orders from Helix 

7)  in order to prepare data for lo dashboard we need to run sql job LXD_Lo_Dashboard on dnaprod 

8)  tables with data about emp

In [62]:
cos_sim(model(['stair'])[0], model(['escalator'])[0])

0.6253435

In [105]:
def train_step(inputs, targets, model, optimizer, loss_function, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        # Pass the inputs through the transformer
        # last word in targets is <end> token
        predictions = model(inputs)
        
        # Calculate the loss
        # first word in targets is the <start> token
        loss = loss_function(targets, predictions)
    
    # Apply the gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
    # Update the loss and accuracy
    train_loss(loss)
    train_accuracy(targets, predictions)


optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Initialize the metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

num_epochs = 10
batch_size = 1

inputs = ['data about helix purchases', 'data about empower transactions']
targets = tf.random.uniform((2, 1), maxval = 512, dtype = tf.int32)

# Training loop
for epoch in range(num_epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
  
    # Iterate over the training dataset
    for batch_number in range(len(inputs) // batch_size + 1):
        train_step(inputs, targets, model, optimizer, loss_function, train_loss, train_accuracy)
      
        # Print training progress every few batches
        if batch_number % 10 == 0:
            print(f'Epoch {epoch + 1} Batch {batch_number} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

Epoch 1 Batch 0 Loss 6.1741 Accuracy 0.0000
Epoch 2 Batch 0 Loss 6.1098 Accuracy 1.0000
Epoch 3 Batch 0 Loss 6.0834 Accuracy 1.0000
Epoch 4 Batch 0 Loss 6.0416 Accuracy 1.0000
Epoch 5 Batch 0 Loss 5.9871 Accuracy 1.0000


KeyboardInterrupt: 